In [1]:
from tabulate import tabulate
import numpy

In [2]:
#Just a function to display the 4x4 matrix, for debugging/checking purposes 
def display(arr):
#Converts int matrix to hex and generally beautifies the code
    res = []
    for i in arr:
        temp = [] 
        for j in i:
            temp.append(hex(j).replace('0x','').zfill(2).upper())
            res.append(temp)
        print(tabulate(res, tablefmt='fancy_grid', stralign= 'center'))
        print("")

In [3]:
#Function to replace matrix values using S-Box 
def replaceSBoxValues(state):
#Initialize empty matrix to put replaced values 
    replaced_state = [[0 for i in range(4)] for j in range(4)] 
    #The array containing S-Box
    sbox = [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 
    0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 
    0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 
    0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 
    0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
    0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
    0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 
    0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 
    0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 
    0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 
    0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 
    0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 
    0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 
    0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 
    0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 
    0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 
    0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 
    0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 
    0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
    0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 
    0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 
    0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 
    0x54, 0xbb, 0x16]
    #Nested loop that sequentially replaces values of all matrix elements
    for i in range (4):
        for j in range (4):
            replaced_state[i][j] = sbox[state[i][j]]
    return replaced_state

In [4]:
#Function to shift rows of the matrix
def shiftRows(state):
    #no shift on first row
    #shift left by 1 on second row
    state[1] = numpy.roll(state[1], -1)
    #shift left by 2 on third row
    state[2] = numpy.roll(state[2], -2)
    #shift left by 3 on fourth row
    state[3] = numpy.roll(state[3], -3)
    return state

#Function to multiply using the gallois field rules 
def field_multiply(mat_val,col_val): 
    #converts matrix and column values from int to bin 
    #splits them into indivudial digits and 
    #reverses the order for polynomial multiplication 
    mat_val = [*bin(mat_val).replace('0b','')] 
    mat_val = [int(mat_val[-i]) for i in range(1,len(mat_val)+1)] 
    col_val = [*bin(col_val).replace('0b','')] 
    col_val = [int(col_val[-i]) for i in range(1,len(col_val)+1)] 
    #Perform polynomial multiplication 
    ans = numpy.polynomial.polynomial.polymul(mat_val,col_val) 
    #Convert from numpy data type to int, also mod 2 the numbers 
    ans = [int(i%2) for i in ans] 
    #Part of code to check if the resultant number does not lie in the field 
    while True: 
        #if number more than 8 bits, 
        if(len(ans)>8): 
            #last power of x is taken 
            if(ans[-1]==1): 
                #x8 is converted to x4+x3+x+1 
                #higher power bit shifted accordingly 
                temp = len(ans)-9 
                ans[0+temp]^=1 
                ans[1+temp]^=1 
                ans[3+temp]^=1 
                ans[4+temp]^=1 
                ans.pop(-1) 
            else: 
                #if highest power is 0, it is just removed 
                ans.pop(-1) 
        else: 
            break 
            #Reversing the ans since the polynomials were reversed in the beggining 
    ans = [int(ans[-i]) for i in range(1,len(ans)+1)] 
    #Converting from the array to an integer 
    ans = int(''.join(map(str,ans)),2) 
    return ans


In [5]:
#Mix columns 
def mix_columns(state): 
# Define the mix columns matrix 
    mix_matrix = [[2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2]]
    # Multiply the state matrix and the mix columns matrix 
    mixed_state = [[0 for i in range(4)] for j in range(4)] 
    for i in range(4): 
        for j in range(4): 
            for k in range(4): 
                    mixed_state[i][j] ^= field_multiply(state[k][j],mix_matrix[i][k]) 
    # Replace each element in the mixed state with its corresponding value in the finite field 
    for i in range(4): 
        for j in range(4): 
            mixed_state[i][j] = mixed_state[i][j] % 256 
    return mixed_state 

    #Add round key 
def addRoundKey(state): 
    #Define the round key as discussed in the lectures 
    round_key = [[0xac, 0x19, 0x28, 0x57], 
    [0x77, 0xfa, 0xd1, 0x5c], 
    [0x66, 0xdc, 0x29, 0x00], 
    [0xf3, 0x21, 0x41, 0x6a]] 
    # XOR each element of the state with the corresponding element in the round key  
    for i in range(4): 
        for j in range(4): 
            state[i][j] ^= round_key[i][j] 
    return state

In [7]:
#Parent AES-128 function to control all the small steps of the code 
def AES(matrix): 
    print("Initial Matrix: ") 
    display(matrix) 
    print("Matrix after replacing s-box values: ") 
    matrix = replaceSBoxValues(matrix) 
    display(matrix) 
    print("Matrix after shifting rows: ") 
    matrix = shiftRows(matrix) 
    display(matrix) 
    print("Matrix after mixing columns: ") 
    matrix = mix_columns(matrix) 
    display(matrix) 
    print("Matrix after adding round key: ") 
    matrix = addRoundKey(matrix) 
    display(matrix) 
    #Applying full 9 Rounds 
    for i in range(8): 
        matrix = addRoundKey(mix_columns(shiftRows(replaceSBoxValues(matrix)))) 
    #Applying Round 10 
    print("After 8 more rounds: ") 
    display(matrix) 
    print("After last round: ") 
    matrix = addRoundKey(shiftRows(replaceSBoxValues(matrix))) 
    display(matrix) 
    return matrix 
 
#Hard coding the input 
input = [[0xea,0x04,0x65,0x85], 
[0x83,0x45,0x5d,0x96], 
[0x5c,0x33,0x98,0xb0], 
[0xf0,0x2d,0xad,0xc5]] 
 
output = AES(input) 


Initial Matrix: 
╒════╤════╤════╤════╕
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
╘════╧════╧════╧════╛

╒════╤════╤════╤════╕
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
╘════╧════╧════╧════╛

╒════╤════╤════╤════╕
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ EA │ 04 │ 65 │ 85 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 83 │ 45 │ 5D │ 96 │
├────┼────┼────┼────┤
│ 5C │ 33 │ 98 │ B0 │
├────┼────┼──